In [ ]:
import hashlib



In [ ]:
password = '12345Qwert'
salt = 'dotnet2023'

# MD5 Hashes (Vulnerable)

In [ ]:
def get_md5_hash(text: str) -> str:
    md5_hash = hashlib.md5()
    md5_hash.update(text.encode('utf-8'))
    hash_result = md5_hash.hexdigest()
    
    return hash_result

def get_md5_salted_hash(text: str, salt: str) -> str:
    salted_text = salt + text
    
    return get_md5_hash(salted_text)
    
    

In [ ]:
hash = get_md5_hash(password)
salted_hash = get_md5_salted_hash(password, salt)

print(f'The md5 hash is: {hash}')
print(f'The salted md5 hash is: {salted_hash}')

# SHA 256 Hashes

In [ ]:
def get_sha256_hash(text: str) -> str:
    return hashlib.sha256(text.encode()).hexdigest()

def get_sha256_salted_hash(text: str, salt: str) -> str:
    salted_text = salt + text
    return get_sha256_hash(salted_text)

In [ ]:
hash = get_sha256_hash(password)
salted_hash = get_sha256_salted_hash(password, salt)

print(f'The sha256 hash is: {hash}')
print(f'The salted sha256 hash is: {salted_hash}')

# References
1. [Salt OWASP](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#salting)
1. [Pepper OWASP](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#peppering)
